In [2]:
import pandas as pd
import sqlite3

In [3]:
with open('discogs_20201201_artists.xml', 'r', encoding='utf-8') as xml_file:
    xml_tree = etree.parse(xml_file)
artists = {}
for i, artist in enumerate(xml_tree.xpath('/artists/artist')):
    artists[artist.find("id").text] = artist.find("name").text

In [4]:
artist_df = pd.DataFrame.from_dict(artists, orient='index', columns=["artist"])
artist_df.reset_index(inplace=True)
artist_df = artist_df.rename(columns={"index": "artist_id"})

In [2]:
conn = sqlite3.connect('discogs.db')
c = conn.cursor()

In [ ]:
c.execute('CREATE TABLE ARTISTS (arist_id number, artist text)')
conn.commit()
artist_df.to_sql('ARTISTS', conn, if_exists='replace', index = False)

In [6]:
from lxml import etree
with open('discogs_20201201_masters.xml', 'r', encoding='utf-8') as xml_file:
    print("Opening")
    xml_tree = etree.parse(xml_file)
    print("Open")

Opening
Open


In [7]:
xml_file.close()

In [4]:
c.execute('CREATE TABLE MASTERS (main_release number, artist_id number, genre text, year number, title text)')

In [5]:
def create_master(conn, m):
    sql = ''' INSERT INTO MASTERS(main_release, artist_id, genre, year, title)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.executemany(sql, m)
    conn.commit()

In [ ]:
masters = []
for i, master in enumerate(xml_tree.xpath('/masters/master')):
    try:
        print(i)
        main = master.find("main_release").text if master.find("main_release").text else '.'
        artist_id = master.find("artists/artist/id").text if master.find("artists/artist/id").text else '.'
        genre = master.find("genres/genre").text if master.find("genres/genre").text else '.'
        year = master.find("year").text if master.find("year").text else '.'
        title = master.find("title").text if master.find("title").text else '.'
        m = (main,artist_id,genre,year,title)
        masters.append(m)
        if i % 100000 == 0:
            create_master(conn,masters)
            masters = []
            print("wrote to db")
    except: 
        pass

In [3]:
conn = sqlite3.connect('discogs.db')
c = conn.cursor()

In [10]:
c.execute('SELECT ARTIST_ID, COUNT(ARTIST_ID) FROM MASTERS WHERE GENRE="Hip Hop" GROUP BY ARTIST_ID ORDER BY COUNT(ARTIST_ID) DESC LIMIT 5')

rows = c.fetchall()

for row in rows:
    print(row)

(194, 3879)
(132084, 113)
(38661, 111)
(21742, 103)
(100752, 99)


it works bless up